#### This is a one-off script, run from ../../data/embeddings to compile pretrained video and finetuned audio embeddings into a single PyTorch read-able file.

#### Since this is a one-off script, the code is horrible because it does not need to be nice.

#### One must write horrible code some times, to be able to focus on writing good code at other times.

In [13]:
import pandas as pd
import torch
import os

DATA_DIR = "../../data/embeddings/tmp"
OUT_PATH = "../../data/embeddings/embeddings_vid_pretrn_aud_ftune.pt"

trn_labels = pd.read_csv(os.path.join(DATA_DIR, "fewshot_trn_labels.csv"))
val_labels = pd.read_csv(os.path.join(DATA_DIR, "fewshot_val_labels.csv"))

aud_trn = torch.load(os.path.join(DATA_DIR, "audio_ftune_trn.pt"))
aud_val = torch.load(os.path.join(DATA_DIR, "audio_ftune_val.pt"))

vid_trn = torch.load(os.path.join(DATA_DIR, "video_pretrn_trn.pt"))
vid_val = torch.load(os.path.join(DATA_DIR, "video_pretrn_val.pt"))

vid_trn_out = torch.zeros(vid_trn["embeddings"].shape)
aud_trn_out = torch.zeros(aud_trn["embeddings"].shape)

vid_val_out = torch.zeros(vid_val["embeddings"].shape)
aud_val_out = torch.zeros(aud_val["embeddings"].shape)

trn_labels_out, val_labels_out = [], []

mapper = {'Couple Romance': 0, 
         'Dance': 1,
         'Heartbreak Romance': 2,
         'Memes': 3,
         'Pranks': 4}

In [14]:
for idx, filename in enumerate(vid_trn["filenames"]):

    index = int(filename[:-4])
    label = mapper[trn_labels[trn_labels["PostIndex"]==index]["Topic"].values[0]]

    aud_idx = aud_trn["filenames"].index(str(index)+".wav")
    vid_trn_out[idx] = vid_trn["embeddings"][idx]
    aud_trn_out[idx] = aud_trn["embeddings"][aud_idx]

    trn_labels_out.append(label)


for idx, filename in enumerate(vid_val["filenames"]):
    
    index = int(filename[:-4])
    label = mapper[val_labels[val_labels["PostIndex"]==index]["Topic"].values[0]]

    aud_idx = aud_val["filenames"].index(str(index)+".wav")
    vid_val_out[idx] = vid_val["embeddings"][idx]
    aud_val_out[idx] = aud_val["embeddings"][aud_idx]

    val_labels_out.append(label)

In [15]:
# Write to disk:x
# vid_trn_out = [440x400]
# aud_trn_out = [440x512]
# vid_val_out = [108x400]
# aud_val_out = [108x512]
# trn_labels = [440]
# val_labels = [108]

data = {"vid_trn": vid_trn_out,
        "aud_trn": aud_trn_out,
        "vid_val": vid_val_out,
        "aud_val": aud_val_out,
        "trn_labels": trn_labels_out,
        "val_labels": val_labels_out}
        
torch.save(data, OUT_PATH)